In [8]:
import pandas as pd
from pandas.api.types import is_numeric_dtype

from utils.prepare_data import read_data, save_data

In [29]:
def get_distribution(s):
    always_unique = ["demography", "seed", "swept_frequencies"]
    if s.name in always_unique:
        x = f"{s.nunique()} unique values."
    elif is_numeric_dtype(s):
        if s.nunique() == 0:
            x = f"{len(s)}x NaN"
        elif s.nunique() == 1:
            x = f"{s.notna().sum()}x {s.loc[s.notna()].unique()[0]}"
        else:
            x = s.describe().to_string()
    elif s.loc[s.notna()].is_unique:
        x = f"{s.nunique()} unique values."
    else:
        if s.nunique() == 1:
            x = f"{s.notna().sum()}x {s.loc[s.notna()].unique()[0]}"
        else:
            x = s.value_counts().to_string()
    return x

In [30]:
def get_columns_report(df):
    lines = []
    for column in sorted(df.columns):
        lines.append(column)
        lines.append(get_distribution(df[column]))
        lines.append("")
    return "\n".join(lines)

In [31]:
def get_df_report(df, name):
    lines = []
    title = f"{name} simulations"
    lines.append(title.upper())
    lines.append("-"*len(title))
    lines.append("")
    lines.append(get_columns_report(df))
    lines.append("")
    return lines

In [62]:
def make_data_report(df):
    lines = []
    constant_cols = df.columns[data.nunique(dropna=False) == 1].tolist()
    title = "CONSTANT PARAMETERS"
    lines.append(title)
    lines.append("-"*len(title))
    lines.append("")
    for col in constant_cols:
        lines.append(col)
        s = df[col]
        lines.append(f"{len(s)}x {s.unique()[0]}")
        lines.append("")
        df = df.drop(col, axis='columns')
    lines.append("")
    modes = {
        sm: df.loc[df.sweep_mode == sm] for sm in sorted(df.sweep_mode.unique())
    }

    lines.extend(get_df_report(df, "All"))
    for mode, this_df in modes.items():
        lines.extend(get_df_report(this_df, mode))
    return "\n".join(lines)

In [63]:
data = read_data(snakemake.input[0])
report_text = make_data_report(data)

In [64]:
print(report_text)

In [47]:
with open(snakemake.output[0], "w") as f:
    f.write(text)